In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction
A company's market values are variable and affected by different factors. The most important factor could be the thoughts of the public. In this task, you are responsible to analyze the effect of public opinion about a company on that company's market values. In this notebook we will try to understand whether negative tweets about companie affect their market prices.

## Data and Approach

In this notebook we'll try to find tweets showing negative sentiments about a few companies and check the timeseries plot of the market value of these companies with respect to number of negative tweets a day.


## Sentiment Analysis
We'll classsify the sentiments of the given tweets to see which tweets are showing a negative sentiment using vaderSentiment.

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

Import the data containing tweets about about comapnies from 2015 to 2020

In [ ]:
company_tweets = pd.read_csv("/kaggle/input/tweets-about-the-top-companies-from-2015-to-2020/Company_Tweet.csv")
tweet = pd.read_csv("/kaggle/input/tweets-about-the-top-companies-from-2015-to-2020/Tweet.csv")

In [ ]:
tweets = pd.merge(company_tweets, tweet, on='tweet_id', how='inner')

In [ ]:
tweets.head()

In [ ]:
sentences = tweets["body"]
sentenceSample = sentences[:10]
sentenceSampleDf=pd.DataFrame(sentenceSample)
sentenceSampleDf

### Threshold for negative tweets
A tweet is classified as a negative tweet only if vaderSentiment scores it above 0.7 on negative sentiment.

In [ ]:
def negativityCheck(sentence):
    if(analyzer.polarity_scores(sentence)["neg"]>0.7):
        return 1
    else:
        return 0

In [ ]:
sentenceSampleDf["negative"]=pd.DataFrame(sentenceSample.apply(negativityCheck).tolist())

In [ ]:
sentenceSampleDf

In [ ]:
sentenceSample.head()

In [ ]:
tweets.shape

In [ ]:
tweets["negative_sentiment"] = sentenceSampleDf["negative"]=pd.DataFrame(sentences.apply(negativityCheck).tolist())

In [ ]:
tweets.head(30)

## Filtering Negative Tweets

We'll filter out the negative tweets after the sentiment analysis and try to see if this corresponds to a significatly lower closing prices on days where the volume of negative tweets were high.

In [ ]:
negativeTweeets = tweets.loc[tweets["negative_sentiment"]==1]

In [ ]:
negativeTweeets

In [ ]:
tweets.ticker_symbol.unique()

In [ ]:
amazonDf = pd.read_csv("../input/stock-market-data/stock_market_data/nasdaq/csv/AMZN.csv")

Well try to see if negative tweets about amazon over the years have had any effect on its stock price,
please note we are only considering number of tweets here, neither is importance of people tweeting, number of retwets likes etc factored in.

In [ ]:
amazonDf.head()

In [ ]:
negativeTweetsAmazon = negativeTweeets.loc[negativeTweeets['ticker_symbol'] =="AMZN"]

In [ ]:
negativeTweetsAmazon.head()

In [ ]:
import datetime
negativeTweetsAmazon['Date'] = pd.to_datetime(negativeTweetsAmazon['post_date'],unit='s').dt.strftime('%d-%m-%Y')

In [ ]:
negativeTweetsAmazon

In [ ]:
amazonInfluence= pd.merge(negativeTweetsAmazon,amazonDf,on="Date",how="inner")

In [ ]:
amazonInfluence.head()

In [ ]:
tweetsOnAmazonDateCount= negativeTweetsAmazon["Date"].value_counts()

In [ ]:
tweetsOnAmazonDateCount = pd.DataFrame(tweetsOnAmazonDateCount)
tweetsOnAmazonDateCount.head()
tweetsOnAmazonDateCount.reset_index(inplace=True)


In [ ]:
tweetsOnAmazonDateCount.head(5)


In [ ]:
tweetsOnAmazonDateCount["count"] = tweetsOnAmazonDateCount["Date"]
tweetsOnAmazonDateCount.drop("Date",axis=1,inplace=True)
tweetsOnAmazonDateCount["Date"] = tweetsOnAmazonDateCount["index"]

In [ ]:
amazonInfluenceByNo= pd.merge(tweetsOnAmazonDateCount,amazonDf,on="Date",how="inner")

In [ ]:
amazonInfluenceByNo.head()

In [ ]:
import matplotlib.pyplot as plt
amazonInfluenceByNo.set_index("index",inplace=True)
amazonInfluenceByNo.loc["28-07-2016"]
ax= amazonInfluenceByNo.loc['28-07-2016':'27-04-2018',"Open"].plot()



In [ ]:
amazonInfluenceByNo["dateTime"] = pd.to_datetime(amazonInfluenceByNo["Date"])
amazonInfluenceByNo = amazonInfluenceByNo.sort_values(by="dateTime")                                                    
bx= amazonInfluenceByNo.loc['22-01-2015':'09-08-2019',"Open"].plot(figsize=(10,5))

amazonInfluenceByNo.head()


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,15))

openingValues= amazonInfluenceByNo["Open"]
closingValues = amazonInfluenceByNo["Close"]
countTweet = amazonInfluenceByNo["count"]

dates = amazonInfluenceByNo["dateTime"]

ax.plot(dates,openingValues,label="open")
ax.plot(dates,closingValues,label="close")
ax.plot(dates,countTweet,label="number of negative Of Tweets")

ax.legend()


## Result
Since the chart shows that number the share price of amazon had kept on increasing , inspite of having at least 1 negative tweet per day, there isin't a considerable differnce between opening and closing prices either on the days of these tweets. Although we cannot conclude that tweets donot afftect share prices, in this case it can be observed that such a small volume cannot affect the share price of a company over a long period.

### Suggestion

This notebook is not completed yet,I am trying to improve the technique, for this specific exercise, please do mention in comments if given technique is erronous or a hint on better technique , thanking you in advance.